<a href="https://colab.research.google.com/github/NIKITA320495/NLP/blob/main/Twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv('Twitter_Data.csv')
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [ ]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)

In [ ]:
df['category'].value_counts()

,count
category,
1.0,13569
0.0,10941
-1.0,7492


In [ ]:
x=df['clean_text']
y=df['category']

In [ ]:
comments=df['clean_text']

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
lemmatizer=WordNetLemmatizer()
corpus=[]
for i in range(len(comments)):
    review=re.sub('[^a-zA-Z]',' ',comments[i])
    review=review.lower()
    review=review.split()
    review=[lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer,one_hot
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
embedding_layer = keras.layers.Embedding(200, 20)

tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences=tokenizer.texts_to_sequences(corpus)

In [ ]:
sequences

[[1,
  77,
  279,
  33,
  1506,
  543,
  862,
  1696,
  1102,
  49,
  8821,
  88,
  62,
  9,
  25,
  836,
  88,
  311,
  2600,
  4746,
  1025],
 [177, 956, 668, 1193, 6, 1],
 [19, 6, 1, 793, 3, 353, 13, 442, 1103, 1, 42, 1, 3803],
 [267,
  154,
  2693,
  54,
  101,
  1,
  103,
  902,
  2978,
  183,
  14938,
  448,
  10877,
  2302,
  8822,
  43,
  515,
  227,
  600],
 [216, 716, 645, 104, 37, 102, 686, 3300, 1, 73],
 [1168, 3073, 14939, 14940, 577, 2235],
 [3441, 190, 2979, 6066, 1213, 5084, 2980, 11, 1, 98],
 [32, 937, 4200, 5, 1, 1741, 2981, 17, 4747],
 [761,
  8,
  2,
  5509,
  93,
  508,
  4748,
  134,
  700,
  1,
  738,
  10,
  455,
  533,
  7594,
  6067,
  10878,
  134,
  10879,
  2184,
  10880],
 [31, 3804, 1],
 [78,
  5,
  509,
  416,
  38,
  902,
  125,
  1815,
  770,
  41,
  2236,
  529,
  534,
  10881,
  19,
  36,
  26,
  17,
  770,
  5,
  2237,
  92,
  1,
  491],
 [186,
  14941,
  4,
  41,
  2601,
  423,
  76,
  5510,
  805,
  6716,
  14942,
  47,
  1,
  937,
  6,
  4455,
  

In [ ]:
voc_size=5000
max_length=max(len(seq)for seq in sequences)
padded_sequences=pad_sequences(sequences,maxlen=max_length,padding='post')
max_length


43

In [ ]:
padded_sequences

array([[   1,   77,  279, ...,    0,    0,    0],
       [ 177,  956,  668, ...,    0,    0,    0],
       [  19,    6,    1, ...,    0,    0,    0],
       ...,
       [   1, 3003,  259, ...,    0,    0,    0],
       [ 638,  869,  258, ...,    0,    0,    0],
       [  54,   11,    1, ...,    0,    0,    0]], dtype=int32)

In [ ]:
padded_sequences.shape

(32002, 43)

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.regularizers import l2
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=max_length))
model.add(LSTM(150))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 43, 100)           500000    
                                                                 
 lstm_3 (LSTM)               (None, 150)               150600    
                                                                 
 dense_3 (Dense)             (None, 1)                 151       
                                                                 
Total params: 650751 (2.48 MB)
Trainable params: 650751 (2.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(padded_sequences,y,validation_split=0.2,epochs=80)

Epoch 1/80
801/801 [==============================] - 35s 41ms/step - loss: 0.3703 - accuracy: 0.3432 - val_loss: 0.2936 - val_accuracy: 0.3418
Epoch 2/80
801/801 [==============================] - 10s 12ms/step - loss: 0.4625 - accuracy: 0.3435 - val_loss: 0.4743 - val_accuracy: 0.3418
Epoch 3/80
801/801 [==============================] - 8s 10ms/step - loss: 0.4221 - accuracy: 0.3543 - val_loss: 0.4721 - val_accuracy: 0.3418
Epoch 4/80
801/801 [==============================] - 7s 8ms/step - loss: 0.4173 - accuracy: 0.3523 - val_loss: 0.4515 - val_accuracy: 0.4331
Epoch 5/80
801/801 [==============================] - 8s 10ms/step - loss: 0.4070 - accuracy: 0.3732 - val_loss: 0.4579 - val_accuracy: 0.3418
Epoch 6/80
801/801 [==============================] - 6s 7ms/step - loss: 0.4118 - accuracy: 0.3617 - val_loss: 0.4728 - val_accuracy: 0.3418
Epoch 7/80
801/801 [==============================] - 7s 9ms/step - loss: 0.4285 - accuracy: 0.3444 - val_loss: 0.4846 - val_accuracy: 0.3418
